In [18]:
# Import libraries
import pandas as pd
import re

In [13]:
# Import the dataset
from datasets import load_dataset
dataset = load_dataset("ai4privacy/pii-masking-300k")

In [2]:
# Inspect the dataset
print(type(dataset))
print(dataset.shape)

<class 'datasets.dataset_dict.DatasetDict'>
{'train': (177677, 9), 'validation': (47728, 9)}


In [3]:
# Split the dataset into two datasets
train = dataset['train']  
validation = dataset['validation']

In [4]:
# Check what type is 'train'
type(train)

datasets.arrow_dataset.Dataset

In [5]:
# Convert the 'train' dataset to a dataframe
train_df = train.to_pandas()

In [6]:
# Check if train_df is a df
type(train_df)

pandas.core.frame.DataFrame

In [7]:
# Inspect the df
train_df.head(10)

,source_text,target_text,privacy_mask,span_labels,mbert_text_tokens,mbert_bio_labels,id,language,set
0,Subject: Group Messaging for Admissions Proces...,Subject: Group Messaging for Admissions Proces...,"[{'value': 'wynqvrh053', 'start': 287, 'end': ...","[[440, 453, ""USERNAME""], [430, 437, ""TIME""], [...","[Sub, ##ject, :, Group, Mess, ##aging, for, Ad...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",40767A,English,train
1,- Meeting at 2:33 PM\n- N23 - Meeting at 11:29...,- Meeting at [TIME]\n- [USERNAME] - Meeting at...,"[{'value': '2:33 PM', 'start': 13, 'end': 20, ...","[[74, 81, ""TIME""], [50, 60, ""USERNAME""], [40, ...","[-, Meeting, at, 2, :, 33, PM, -, N, ##23, -, ...","[O, O, O, B-TIME, I-TIME, I-TIME, I-TIME, O, O...",40767B,English,train
2,Subject: Admission Notification - Great Britai...,Subject: Admission Notification - Great Britai...,"[{'value': '5:24am', 'start': 263, 'end': 269,...","[[395, 407, ""SOCIALNUMBER""], [358, 375, ""EMAIL...","[Sub, ##ject, :, Ad, ##mission, Not, ##ificati...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",40768A,English,train
3,Card: KB90324ER\n Country: GB\n Building: ...,Card: [IDCARD]\n Country: [COUNTRY]\n Buil...,"[{'value': 'KB90324ER', 'start': 6, 'end': 15,...","[[390, 393, ""STATE""], [368, 378, ""CITY""], [346...","[Card, :, KB, ##90, ##32, ##4, ##ER, \, n, Cou...","[O, O, B-IDCARD, I-IDCARD, I-IDCARD, I-IDCARD,...",40768B,English,train
4,"N, WA14 5RW\n Password: r]iD1#8\n\n...and so...","N, WA14 5RW\n Password: [PASS]\n\n...and so ...","[{'value': 'r]iD1#8', 'start': 26, 'end': 33, ...","[[336, 352, ""DATE""], [26, 33, ""PASS""]]","[N, ,, W, ##A, ##14, 5, ##R, ##W, \, n, Pass, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-PASS...",40768C,English,train
5,Subject: Admission Application Attachments Con...,Subject: Admission Application Attachments Con...,"[{'value': '301025226', 'start': 311, 'end': 3...","[[311, 320, ""PASSPORT""]]","[Sub, ##ject, :, Ad, ##mission, Application, A...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",40769A,English,train
6,981\n- Social Security Number: 554.575.9355\n-...,981\n- Social Security Number: [SOCIALNUMBER]\...,"[{'value': '554.575.9355', 'start': 30, 'end':...","[[263, 279, ""TEL""], [226, 249, ""EMAIL""], [206,...","[981, -, Social, Security, Number, :, 554, ., ...","[O, O, O, O, O, O, O, O, B-SOCIALNUMBER, I-SOC...",40769B,English,train
7,s carefully and inform us immediately if there...,s carefully and inform us immediately if there...,"[{'value': 'Rue des Écoles', 'start': 320, 'en...","[[342, 354, ""DATE""], [338, 340, ""TIME""], [320,...","[s, care, ##fully, and, info, ##rm, us, immedi...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",40769C,English,train
8,"- id_1:\n Feb 8, 1986\n iloweintögl\n 4929-...",- id_1:\n [BOD]\n [USERNAME]\n [TEL]\n Det...,"[{'value': 'Feb 8, 1986', 'start': 10, 'end': ...","[[287, 300, ""TEL""], [280, 284, ""USERNAME""], [2...","[-, id, _, 1, :, Feb, 8, ,, 1986, il, ##owe, #...","[O, O, O, O, O, O, B-BOD, I-BOD, I-BOD, I-BOD,...",40772A,English,train
9,y involved in community service initiatives.\n...,y involved in community service initiatives.\n...,"[{'value': 'Apr 29, 1973', 'start': 55, 'end':...","[[339, 355, ""TEL""], [314, 336, ""USERNAME""], [2...","[y, involved, in, community, service, initiati...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-BOD,...",40772B,English,train


In [ ]:
pd.set_option('display.max_colwidth', None)
train_df[76521:76541]

label: mbert_bio_labels
features: source_text

In [10]:
# Use a pipeline as a high-level helper
import torch as pt
from transformers import pipeline

pipe = pipeline("token-classification", model="lakshyakh93/deberta_finetuned_pii", framework='pt')

In [11]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("lakshyakh93/deberta_finetuned_pii")
model = AutoModelForTokenClassification.from_pretrained("lakshyakh93/deberta_finetuned_pii")

In [25]:
# Basic regex model to identify ssn
ssn_pattern = r'\b\d{3} \d{3} \d{4}\b'
text = "The patient's SSN is 123 456 6789."
ssn_matches = re.findall(ssn_pattern, text)
print(ssn_matches)

['123 456 6789']


In [26]:
# Function to redact SSN in text
def redact_ssn(text): 
    redacted_text = re.sub(ssn_pattern, '["SOCIALNUMBER"]', text)
    return redacted_text

# Apply the regex to the 'Text' column to find SSNs
train_df['SSN_found'] = train_df['source_text'].apply(redact_ssn)

In [27]:
train_df

,source_text,target_text,privacy_mask,span_labels,mbert_text_tokens,mbert_bio_labels,id,language,set,SSN_found
0,"Subject: Group Messaging for Admissions Process\n\nGood morning, everyone,\n\nI hope this message finds you well. As we continue our admissions processes, I would like to update you on the latest developments and key information. Please find below the timeline for our upcoming meetings:\n\n- wynqvrh053 - Meeting at 10:20am\n- luka.burg - Meeting at 21\n- qahil.wittauer - Meeting at quarter past 13\n- gholamhossein.ruschke - Meeting at 9:47 PM\n- pdmjrsyoz1460","Subject: Group Messaging for Admissions Process\n\nGood morning, everyone,\n\nI hope this message finds you well. As we continue our admissions processes, I would like to update you on the latest developments and key information. Please find below the timeline for our upcoming meetings:\n\n- [USERNAME] - Meeting at [TIME]\n- [USERNAME] - Meeting at [TIME]\n- [USERNAME] - Meeting at [TIME]\n- [USERNAME] - Meeting at [TIME]\n- [USERNAME]","[{'value': 'wynqvrh053', 'start': 287, 'end': 297, 'label': 'USERNAME'}, {'value': '10:20am', 'start': 311, 'end': 318, 'label': 'TIME'}, {'value': 'luka.burg', 'start': 321, 'end': 330, 'label': 'USERNAME'}, {'value': '21', 'start': 344, 'end': 346, 'label': 'TIME'}, {'value': 'qahil.wittauer', 'start': 349, 'end': 363, 'label': 'USERNAME'}, {'value': 'quarter past 13', 'start': 377, 'end': 392, 'label': 'TIME'}, {'value': 'gholamhossein.ruschke', 'start': 395, 'end': 416, 'label': 'USERNAME'}, {'value': '9:47 PM', 'start': 430, 'end': 437, 'label': 'TIME'}, {'value': 'pdmjrsyoz1460', 'start': 440, 'end': 453, 'label': 'USERNAME'}]","[[440, 453, ""USERNAME""], [430, 437, ""TIME""], [395, 416, ""USERNAME""], [377, 392, ""TIME""], [349, 363, ""USERNAME""], [344, 346, ""TIME""], [321, 330, ""USERNAME""], [311, 318, ""TIME""], [287, 297, ""USERNAME""]]","[Sub, ##ject, :, Group, Mess, ##aging, for, Ad, ##mission, ##s, Process, Good, morning, ,, everyone, ,, I, hope, this, message, finds, you, well, ., As, we, continue, our, admission, ##s, processes, ,, I, would, like, to, update, you, on, the, latest, developments, and, key, information, ., Please, find, below, the, time, ##line, for, our, upcoming, meetings, :, -, w, ##yn, ##q, ##vr, ##h, ##0, ##53, -, Meeting, at, 10, :, 20, ##am, -, luka, ., bu, ##rg, -, Meeting, at, 21, -, q, ##ahi, ##l, ., wit, ##tau, ##er, -, Meeting, at, quarter, past, 13, -, gh, ##ola, ##mh, ##osse, ...]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-USERNAME, I-USERNAME, I-USERNAME, O, O, O, O, O, O, O, B-TIME, I-TIME, I-TIME, O, B-USERNAME, I-USERNAME, O, O, O, B-TIME, I-TIME, I-USERNAME, I-USERNAME, I-USERNAME, ...]",40767A,English,train,"Subject: Group Messaging for Admissions Process\n\nGood morning, everyone,\n\nI hope this message finds you well. As we continue our admissions processes, I would like to update you on the latest developments and key information. Please find below the timeline for our upcoming meetings:\n\n- wynqvrh053 - Meeting at 10:20am\n- luka.burg - Meeting at 21\n- qahil.wittauer - Meeting at quarter past 13\n- gholamhossein.ruschke - Meeting at 9:47 PM\n- pdmjrsyoz1460"
1,"- Meeting at 2:33 PM\n- N23 - Meeting at 11:29pm\n- wennmann27 - Meeting at 4:45 PM\n\nKindly make sure to adjust your schedules accordingly. Our group messaging platform will be utilized for any updates, reminders, or changes to the agenda. This collaborative approach will ensure smooth communication and coordination throughout the admissions process.\n\nLooking forward to our productive discussions and successful collaboration.\n\nWarm regards,\n[Your Name]","- Meeting at [TIME]\n- [USERNAME] - Meeting at [TIME]\n- [USERNAME] - Meeting at [TIME]\n\nKindly make sure to adjust your schedules accordingly. Our group messaging 